In [15]:
!git add .
!git commit -m "dataset of mvoies"
!git push

[main dc8062d] dataset of mvoies
 23 files changed, 60667 insertions(+)
 create mode 100644 .config/.last_opt_in_prompt.yaml
 create mode 100644 .config/.last_survey_prompt.yaml
 create mode 100644 .config/.last_update_check.json
 create mode 100644 .config/active_config
 create mode 100644 .config/config_sentinel
 create mode 100644 .config/configurations/config_default
 create mode 100644 .config/default_configs.db
 create mode 100644 .config/gce
 create mode 100644 .config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db
 create mode 100644 .config/logs/2026.01.16/14.23.31.981136.log
 create mode 100644 .config/logs/2026.01.16/14.24.03.314209.log
 create mode 100644 .config/logs/2026.01.16/14.24.13.071214.log
 create mode 100644 .config/logs/2026.01.16/14.24.18.954466.log
 create mode 100644 .config/logs/2026.01.16/14.24.28.646070.log
 create mode 100644 .config/logs/2026.01.16/14.24.29.392089.log
 create mode 100755 sample_data/README.md
 create mode 100755 sample_dat

In [166]:
!pip install nltk

In [192]:
import numpy as np
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [128]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [129]:
movies.head(1)
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [130]:
#merging the two datasets we have
movies = movies.merge(credits , on='title')

In [131]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [132]:
#feature selection
# selected features(by me ) -> genres , id , keywords , title , overview  , popularity ,  production_companies , producation_countries , runtime , vote_average , cast , crew
# selected features(by codes) -> genres , id , keywords , title , overview , cast , crew (isne sirf text walle liye h numerical hata diye Qki iske flow ke sath nahi jaa rhe the vo)
# movies_byme = movies[['genres',
#     'id',
#     'keywords',
#     'title',
#     'overview',
#     'popularity',
#     'production_companies',
#     'production_countries',
#     'runtime',
#     'vote_average',
#     'cast',
#     'crew']]
movies = movies[['genres',
    'id',
    'keywords',
    'title',
    'overview',
    'cast',
    'crew']]

In [133]:
movies.head(1)

,genres,id,keywords,title,overview,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [134]:
movies.isnull().sum()

,0
genres,0
id,0
keywords,0
title,0
overview,3
cast,0
crew,0


In [135]:
movies_byme.dropna(inplace=True)

In [136]:
movies.duplicated().sum()

np.int64(0)

In [137]:
#initially all the columns are in the form of dict within a string of list we need to make a helper function to extract them
#usse phle string of list se list me convert krna padhega using lib "ast" lib and literal_eval() function
def convert(obj):
  l = []
  for i in ast.literal_eval(obj):
    l.append(i['name'])
  return l

In [138]:
#applying the convert function
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

In [139]:
# hame cast se top 3
def convertTop3(obj):
  cnt = 0
  l = []
  for i in ast.literal_eval(obj):
    if cnt != 3:
      l.append(i['name'])
      cnt += 1
    else:
      break
  return l

In [140]:
movies['cast'] = movies['cast'].apply(convertTop3)

In [141]:
# now fetching the director of the movie
def fetch_director(obj):
  l = []
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director':
      l.append(i['name'])
      break
  return l

In [142]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [143]:
# movi8es overview walla col bhi ek string isko bhi ek list me convert krenge
movies['overview'] = movies['overview'].fillna('')
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [144]:
# sabb merge krne se phle hame names/words ke beech ka gap hatana padhega to remove the confusion with same names
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

In [145]:
movies.head()

,genres,id,keywords,title,overview,cast,crew
0,"[Action, Adventure, Fantasy, ScienceFiction]",19995,"[cultureclash, future, spacewar, spacecolony, ...",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,"[Adventure, Fantasy, Action]",285,"[ocean, drugabuse, exoticisland, eastindiatrad...",Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,"[Action, Adventure, Crime]",206647,"[spy, basedonnovel, secretagent, sequel, mi6, ...",Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,"[Action, Crime, Drama, Thriller]",49026,"[dccomics, crimefighter, terrorist, secretiden...",The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,"[Action, Adventure, ScienceFiction]",49529,"[basedonnovel, mars, medallion, spacetravel, p...",John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [146]:
#mergings columns to make tags
# we are merging genres , overview ,  keywords ,  cast(only top 1) , crew(only director)
movies['tags'] = movies['genres'] + movies['overview'] + movies['keywords'] + movies['cast'] + movies['crew']

In [147]:
movies.head()

,genres,id,keywords,title,overview,cast,crew,tags
0,"[Action, Adventure, Fantasy, ScienceFiction]",19995,"[cultureclash, future, spacewar, spacecolony, ...",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, ScienceFiction, I..."
1,"[Adventure, Fantasy, Action]",285,"[ocean, drugabuse, exoticisland, eastindiatrad...",Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Adventure, Fantasy, Action, Captain, Barbossa..."
2,"[Action, Adventure, Crime]",206647,"[spy, basedonnovel, secretagent, sequel, mi6, ...",Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[Action, Adventure, Crime, A, cryptic, message..."
3,"[Action, Crime, Drama, Thriller]",49026,"[dccomics, crimefighter, terrorist, secretiden...",The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[Action, Crime, Drama, Thriller, Following, th..."
4,"[Action, Adventure, ScienceFiction]",49529,"[basedonnovel, mars, medallion, spacetravel, p...",John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[Action, Adventure, ScienceFiction, John, Cart..."


In [150]:
new_df = movies[['id' , 'title' , 'tags']]

In [151]:
new_df

,id,title,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction, I..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action, Captain, Barbossa..."
2,206647,Spectre,"[Action, Adventure, Crime, A, cryptic, message..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller, Following, th..."
4,49529,John Carter,"[Action, Adventure, ScienceFiction, John, Cart..."
...,...,...,...
4804,9367,El Mariachi,"[Action, Crime, Thriller, El, Mariachi, just, ..."
4805,72766,Newlyweds,"[Comedy, Romance, A, newlywed, couple's, honey..."
4806,231617,"Signed, Sealed, Delivered","[Comedy, Drama, Romance, TVMovie, ""Signed,, Se..."
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [153]:
#convert again into s string
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

/tmp/ipython-input-3089450492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [161]:
# comverting all into lowercase
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

/tmp/ipython-input-2955537219.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [203]:
# isko vector me convert krna padhega Qki text ko compare krna h hame
# hamm sbse phle ek table bana lenge jisme colmns honge words and rows honge movies and values hongi kitni baar konsa word use hua uss specific movie me
# ye krte time we will not consider the stop words ( vo words jinka scentence ke meaning me koi kaam nhi hota h)
# vectoriazation ke liye ham "sklearn.feature_extraction.text.CountVectorizer" use krenge
cv = CountVectorizer(max_features=10000 , stop_words='english')

In [204]:
# by default sparce matrix return krta h to usko array me convert krnege
vectors = cv.fit_transform(new_df['tags']).toarray()

In [205]:
# tfidf = TfidfVectorizer(
#     max_features=5000,
#     stop_words='english'
# )

# vectors = tfidf.fit_transform(new_df['tags']).toarray()

In [206]:
ps = PorterStemmer()

In [207]:
# abhi jo data milla h ye sb krne ke baad usme similar words alg alg h jaise ki ( action/actions , activities/activity)
# issse shi krne ke liye nltk lib use krenge
def stem(text):
  y = []
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [208]:
new_df['tags'] = new_df['tags'].apply(stem)

/tmp/ipython-input-3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [209]:
# abhi hamne cosin distance between the vectors
similarity = cosine_similarity(vectors)

In [210]:
from scipy.spatial import distance
# abhi ek function banana h jisme ham aagar koi movie de to co best 5 recomended movies return krke de hame
# jb distance nikal kr sort krnege to ek issue aayega ki index value loose krr denegi movies to issko solve krne ke liye enumerate function ka use krenge and uska list bana lenge
def recommend(movie):
  movie_index = new_df[new_df['title'] == movie].index[0]
  distances = similarity[movie_index]
  movies_list  = sorted(list(enumerate(distances)) , reverse=True , key=lambda x:x[1])[1:6]

  for i in movies_list:
    print(new_df.iloc[i[0]].title)
  for i in movies_list:
    print(new_df.iloc[i[0]].title, "->", i[1])

In [211]:
recommend('Avatar')

Aliens
Aliens vs Predator: Requiem
Titan A.E.
Falcon Rising
Battle: Los Angeles
Aliens -> 0.24043316991025832
Aliens vs Predator: Requiem -> 0.24016144275253304
Titan A.E. -> 0.22514075697341696
Falcon Rising -> 0.22281245492773066
Battle: Los Angeles -> 0.21252319110217674


In [213]:
import pickle

In [214]:
pickle.dump(new_df.to_dict() , open('movies_dict.pkl' , 'wb'))

In [216]:
pickle.dump(similarity , open('similarity.pkl' , 'wb'))